In [2]:
import sys
# 直接导入Smote是不行的，必须将路径加入才可以
sys.path.append("C:Users\perma\PycharmProjects\paper")
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from algorithm.SMOTE import Smote

In [3]:
# 读取数据
column_names = ["Id", "RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe", "class"]
data_path = "C:\\Users\\perma\\PycharmProjects\\paper\\data\\glass.data"
table = pd.read_table(filepath_or_buffer=data_path,
                      sep=",",
                      header=None,
                      index_col=None,
                      names=column_names)
attributes = ["RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe"]
data = table[attributes]
label = table["class"]
print("length: data, label", len(data), len(label))
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(X=data)
scaled_data[0:3]
minor_class = [5, 6, 7]
major_class = [1, 2, 3, 4]
new_label = []
for l in label:
    if l in major_class:
        new_label.append(0.0)
    else:
        new_label.append(1.0)


length: data, label 214 214


D:\software\anaconda\exe\envs\forpaper\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: read_table is deprecated, use read_csv instead.
  


In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, f1_score
from sklearn.model_selection import train_test_split

In [5]:
knn = KNeighborsClassifier()
dc = DecisionTreeClassifier()
svm = SVC()
lg = LogisticRegression()
nb = GaussianNB()
algorithm = {"knn": knn, "tree": dc, "svm": svm, "lgs": lg, "nb": nb}
alg_params = {"knn": {"n_neighbors": 7}, 
              "tree": {}, 
              "svm": {}, 
              "lgs": {}, 
              "nb": {}}

In [17]:
# 将少数类和多数类分开提取出来
major = []
major_label = []
minor = []
minor_label = []
for label_index in range(len(new_label)):
    if new_label[label_index] == 1.0:
        minor.append(scaled_data[label_index, :])
        minor_label.append(new_label[label_index])
    else:
        major.append(scaled_data[label_index, :])
        major_label.append(new_label[label_index])
# 定义gmean函数
def g_mean(ground_truth, prediction):
    matrix = confusion_matrix(y_true=ground_truth, y_pred=prediction)
    tpr = matrix[1, 1] / (matrix[1, 1] + matrix[1, 0])
    tpn = matrix[0, 0] / (matrix[0, 0] + matrix[0, 1])
    return np.sqrt(tpr * tpn)
result = dict([(key, {"gmean": list(), "fscore": list()}) for key in algorithm.keys()])
# 实验次数
n=100
for i in range(n):
    for key in algorithm.keys():
        alg = algorithm[key]
        alg.set_params(**alg_params[key])
        major_train, major_test, \
        major_label_train, major_label_test \
            = train_test_split(major, major_label, shuffle=True, test_size=0.25)
        minor_train, minor_test, \
        minor_label_train, minor_label_test \
            = train_test_split(minor, minor_label, shuffle=True, test_size=0.25)
        data_train = np.concatenate((major_train, minor_train), axis=0)
        label_train = np.concatenate((major_label_train, minor_label_train), axis=0)
        data_test = np.concatenate((major_test, minor_test), axis=0)
        label_test = np.concatenate((major_label_test, minor_label_test), axis=0)
        # smote生成合成样本
        minor_samples = []
        for label_index in range(len(label_train)):
            if label_train[label_index] == 1.0:
                minor_samples.append(data_train[label_index])
        smote = Smote(minor_samples, 300, 15)
        smote.over_sampling()
        syntheticed_samples = smote.synthetic
        syntheticed_labels = len(syntheticed_samples) * [1.0]
        syntheticed_train = np.concatenate((data_train, syntheticed_samples), axis=0)
        syntheticed_train_label = np.concatenate((label_train, syntheticed_labels), axis=0)
        alg.fit(X=syntheticed_train, y=syntheticed_train_label)
        prediction = alg.predict(X=data_test)
        gmean = g_mean(label_test, prediction)
        fscore = f1_score(y_true=label_test, y_pred=prediction) / 2
        result[key]["gmean"].append(gmean)
        result[key]["fscore"].append(fscore)
new_line = 0       
for alg_key in result.keys():
    new_line += 1
    alg = result[alg_key]
    for score_key in alg.keys():
        mean_score = np.mean(alg[score_key])
        print(alg_key, " : ", score_key, " : ", mean_score, end="-----------------")
    print("\n")


knn  :  gmean  :  0.9369522075078404-----------------knn  :  fscore  :  0.4401948048487242-----------------

tree  :  gmean  :  0.9035464543511881-----------------tree  :  fscore  :  0.426075608223673-----------------

svm  :  gmean  :  0.860092962519215-----------------svm  :  fscore  :  0.39347987626866504-----------------

lgs  :  gmean  :  0.8873335120783972-----------------lgs  :  fscore  :  0.411498088555913-----------------

nb  :  gmean  :  0.8760190743847386-----------------nb  :  fscore  :  0.4130698440295892-----------------



In [1]:
# # k = 3
# knn  :  gmean  :  0.9314632519167819-----------------knn  :  fscore  :  0.4365387389914086-----------------
# 
# tree  :  gmean  :  0.8837708029008948-----------------tree  :  fscore  :  0.41675847414377665-----------------
# 
# svm  :  gmean  :  0.8708426190490793-----------------svm  :  fscore  :  0.39961581623073333-----------------
# 
# lgs  :  gmean  :  0.8896021989982745-----------------lgs  :  fscore  :  0.4088651281915464-----------------
# 
# nb  :  gmean  :  0.884368213674143-----------------nb  :  fscore  :  0.4112518505212096-----------------

In [8]:
# # K = 4
# knn  :  gmean  :  0.93545970240799-----------------knn  :  fscore  :  0.4372203822940336-----------------
# 
# tree  :  gmean  :  0.8962134969201729-----------------tree  :  fscore  :  0.4225824195535585-----------------
# 
# # svm  :  gmean  :  0.86545986876768-----------------svm  :  fscore  :  0.3989673586348201-----------------
# 
# lgs  :  gmean  :  0.8992093023726833-----------------lgs  :  fscore  :  0.41652964572993356-----------------
# 
# nb  :  gmean  :  0.8940945276678378-----------------nb  :  fscore  :  0.41872163900969817-----------------

In [10]:
# # k = 5
# knn  :  gmean  :  0.9384987956511892-----------------knn  :  fscore  :  0.44214686061888514-----------------
# 
# tree  :  gmean  :  0.8940841151936233-----------------tree  :  fscore  :  0.4234206022741907-----------------
# 
# svm  :  gmean  :  0.872779050695092-----------------svm  :  fscore  :  0.40221301722535946-----------------
# 
# lgs  :  gmean  :  0.8908065437127949-----------------lgs  :  fscore  :  0.41274401307782116-----------------
# 
# nb  :  gmean  :  0.884817928999969-----------------nb  :  fscore  :  0.41472335925360665-----------------


In [12]:
# #  k =6
# knn  :  gmean  :  0.9287508548389419-----------------knn  :  fscore  :  0.4336866008555282-----------------
# 
# tree  :  gmean  :  0.8998187057020596-----------------tree  :  fscore  :  0.4245226295869518-----------------
# 
# svm  :  gmean  :  0.8645014139020877-----------------svm  :  fscore  :  0.3976084039016168-----------------
# 
# lgs  :  gmean  :  0.8941887742427105-----------------lgs  :  fscore  :  0.41381901642649277-----------------
# 
# nb  :  gmean  :  0.8957115416845669-----------------nb  :  fscore  :  0.4229753352462685-----------------

In [14]:
# # k =7
# # knn  :  gmean  :  0.9426433947656854-----------------knn  :  fscore  :  0.44562979247506507-----------------
# # 
# # tree  :  gmean  :  0.8842879067067304-----------------tree  :  fscore  :  0.4174324302048753-----------------
# # 
# # svm  :  gmean  :  0.8699231838075978-----------------svm  :  fscore  :  0.40029899748670844-----------------
# # 
# # lgs  :  gmean  :  0.891793765906231-----------------lgs  :  fscore  :  0.4127127413094431-----------------
# # 
# # nb  :  gmean  :  0.8817630587824777-----------------nb  :  fscore  :  0.41358522721206553-----------------

In [15]:
# # k =1 
# knn  :  gmean  :  0.9480193683905904-----------------knn  :  fscore  :  0.44587653950507117-----------------
# 
# tree  :  gmean  :  0.8928913103275136-----------------tree  :  fscore  :  0.4255872793521712-----------------
# 
# svm  :  gmean  :  0.8695320186695361-----------------svm  :  fscore  :  0.4020636049008999-----------------
# 
# lgs  :  gmean  :  0.8898945067381304-----------------lgs  :  fscore  :  0.4105389364046658-----------------
# 
# nb  :  gmean  :  0.8703926043617861-----------------nb  :  fscore  :  0.40524646437263706-----------------


In [18]:
# # k =8 
# knn  :  gmean  :  0.9378560990403381-----------------knn  :  fscore  :  0.44178723144414195-----------------
# 
# tree  :  gmean  :  0.8958288977716704-----------------tree  :  fscore  :  0.42219978007998477-----------------
# 
# svm  :  gmean  :  0.8669944036249574-----------------svm  :  fscore  :  0.3990487046825799-----------------
# 
# lgs  :  gmean  :  0.8875755665331814-----------------lgs  :  fscore  :  0.4121753407937018-----------------
# 
# nb  :  gmean  :  0.891633523290444-----------------nb  :  fscore  :  0.4190910283619428-----------------

In [ ]:
# # k = 15
# knn  :  gmean  :  0.9369522075078404-----------------knn  :  fscore  :  0.4401948048487242-----------------
# 
# tree  :  gmean  :  0.9035464543511881-----------------tree  :  fscore  :  0.426075608223673-----------------
# 
# svm  :  gmean  :  0.860092962519215-----------------svm  :  fscore  :  0.39347987626866504-----------------
# 
# lgs  :  gmean  :  0.8873335120783972-----------------lgs  :  fscore  :  0.411498088555913-----------------
# 
# nb  :  gmean  :  0.8760190743847386-----------------nb  :  fscore  :  0.4130698440295892-----------------
